In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, mne, json, h5py, pymatreader, re, time, gc;
import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mpr
import matplotlib as mpl
import scipy.signal as sig
import pandas as pd 
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

from dateutil import parser
from utils_postprocess_HPC import collectCalcResults

light_only = 1

# 30 jan was for the first submitted manuscript
start_time = parser.parse("30 jan 2023 00:05:00")

#end_time = parser.parse("30 Oct 2021 21:21:45")
#end_time = parser.parse("30 Oct 2029 21:21:45")
end_time = parser.parse("15 June 2023 21:21:45")


pairs = []
#pairs += [('covmat_entire','per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_entire')]
pairs += [('covmat_rest',  'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest')]
pairs += [('SSS_covmat_rest',  'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_SSS_covmat_rest')]

#per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest_noscaling
# pairs += [('covmat_entire_noscale',
#             'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_entire_noscaling')]
# pairs += [('covmat_rest_noscale',  
#             'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest_noscaling')]

In [ ]:
copied_to_scratch = False

In [ ]:
#copied_to_scratch = True

# copy to scratch

In [ ]:
gv.data_dir_tmp,subdir_cur

In [ ]:
gv.data_dir_tmp

In [ ]:
pjoin(gv.data_dir_tmp,subdir_cur)

In [ ]:
import shutil
scratch_copy = True
copied_to_scratch = False
if scratch_copy:
    for pptype,subdir_cur in pairs:
        print(pptype,subdir_cur)
        import subprocess as sp
        if not os.path.exists(pjoin(gv.data_dir_tmp,subdir_cur)):
            os.makedirs(pjoin(gv.data_dir_tmp,subdir_cur))
        #out = sp.getoutput(f'cp -r {pjoin(gv.data_dir,subdir_cur)}/_!_* {pjoin(gv.data_dir_tmp,subdir_cur)}')
        
        s = f'rsync -tv {pjoin(gv.data_dir,subdir_cur)}/_!_* {pjoin(gv.data_dir_tmp,subdir_cur)}/'
        #print(s)
        out = sp.getoutput(s)
        #shutil.copytree(pjoin(gv.data_dir,subdir_cur), 
        #                pjoin(gv.data_dir_tmp,subdir_cur) )
        print(out)
    copied_to_scratch = True
else:
    print('Skipping scratch copy')

In [ ]:
perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
to_show += [('trem_vs_2class','merge_movements','basic')]
to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]

In [ ]:
end_time = parser.parse("15 June 2023 21:21:45")

In [ ]:
#test
from utils_postprocess_HPC import prepTableInfo3
copied_to_scratch = True
use_tmpdir_to_load = copied_to_scratch
use_tmpdir_to_load = copied_to_scratch
prefs_test = ['onlyH_act_subskip8@both-copy_from_search_LFP']
dftest, table_info_per_perf_type, table_per_perf_type = \
    prepTableInfo3(opr, prefixes=prefs_test, 
    perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True,
    use_tmpdir_to_load = use_tmpdir_to_load)

In [ ]:
import gc; gc.collect()

In [ ]:
out

# Load

In [ ]:
use_tmpdir_to_load = copied_to_scratch
pptype2res = {}
for pptype,subdir_cur in pairs:
    print(pptype,subdir_cur)
    output_per_raw_cur  = collectCalcResults(subdir_cur, start_time =start_time,
                end_time=end_time,
                use_tmpdir_to_load = use_tmpdir_to_load)
    pptype2res[pptype]  = output_per_raw_cur
    
opr = output_per_raw_cur
from utils_postprocess import  multiLevelDict2TupleList
outputs_grouped_tpll = multiLevelDict2TupleList(opr,4,3)
outputs_grouped_tpll[0][:-1]
dfunproc = pd.DataFrame(outputs_grouped_tpll, 
    columns = ['rawname','prefix','grouping','interval_set','mcf'] )
prefixes = list( dfunproc['prefix'].unique() )

In [ ]:
dfunproc.iloc[100]['mcf'].keys()

In [ ]:
'mcf' in dfunproc.iloc[100]

In [ ]:
dfunproc['_mtime'] = dfunproc.apply(lambda x: x['mcf']['mod_time_datetime'] if 'mcf' in x else None,1 )

In [ ]:
dfunproc['_mtime'].max()

In [ ]:
list (sorted( prefixes ))

### dbg

In [ ]:
from utils_postprocess_HPC import getMocFromRow
rcd = dfunproc.to_dict(orient='records')[0]
#getMocFromRow(rcd, opr)#['pars']

In [ ]:
if not nloaded:
    print('Nothing is loaded :(')
# load large file with gathered
import utils_postprocess as pp
load_precollected = 0
if load_precollected:
    output_per_raw = np.load(pjoin(gv.data_dir,subdir,'gathered.npz'),allow_pickle=1)
    output_per_raw = output_per_raw['output_per_raw'][()]
    print('len(output_per_raw) =', len(output_per_raw))
    tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)

    z0 = [tpl[:-1] for tpl in tpll]
    #rns_ord, prefs_ord, grp_ord, it_ord = list (zip(*z0  ) )
    tpll_reshaped = np.array( list (zip(*z0  ) ) )

### prepare table info (long)

In [ ]:
#%debug
# mult_clf_output = tpll[0][-1]
# all_thrs = mult_clf_output['feat_variance_q_thr'][-1:]
# #thr0,thr1,thr2='0.87','0.92','0.99'
# all_LDA =  []
# all_XGB = ['all_present_features'] #,'after_VF_threshold']
# for thr_cur in all_thrs:
#     all_XGB += [ f'best_PCA-derived_features_{thr_cur}']

# perf_to_use_list = []
# for v in all_XGB[1:]:
#     perf_to_use_list += [('XGB',all_XGB[0],v,'across_medcond')]
    #perf_to_use_list += [('XGB',all_XGB[0],v,'across_subj')]
    
#perf_to_use_list = [('XGB',all_XGB[0], f'best_PCA-derived_features_{all_thrs[-1]}','across_subj')]
#perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]
perf_to_use_list = [('XGB','all_present_features', None)]

###########################################
display(perf_to_use_list)

# to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
#         ('trem_vs_2class','merge_movements','basic')]
#to_show = [('trem_vs_mvt','merge_movements','trem_vs_hold&move'),
#           ('trem_vs_all','merge_all_not_trem','basic') ]
#             ('trem_vs_2class','merge_movements','basic'),
#to_show = [('trem_vs_all','merge_all_not_trem','basic') ]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
to_show += [('trem_vs_2class','merge_movements','basic')]
to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]
# to_show += [('trem_vs_quiet','merge_all_not_trem','trem_vs_quiet')00 ]
#             ('trem_vs_2class','merge_movements','basic'),
from utils_postprocess import  multiLevelDict2TupleList

#prefixes = ["onlyH_act_only21"]

use_tmpdir_to_load = copied_to_scratch

dfs = []
for pptype,opr in pptype2res.items():
    outputs_grouped_tpll = multiLevelDict2TupleList(opr,4,3)
    outputs_grouped_tpll[0][:-1]
    dfunproc = pd.DataFrame(outputs_grouped_tpll, 
        columns = ['rawname','prefix','grouping','interval_set','mcf'] )
    prefixes = list( dfunproc['prefix'].unique() )

    # warnings.simplefilter('error')
    dftmp, table_info_per_perf_type, table_per_perf_type = \
        postp.prepTableInfo3(opr, prefixes=prefixes, 
        perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True,
        use_tmpdir_to_load = use_tmpdir_to_load)
    gc.collect()
    dftmp['pptype'] = pptype
    
    dfs += [dftmp]

#df_all = df
df_noext = pd.concat(dfs, ignore_index=True)
import gc; gc.collect()

## make and extendeding tables

In [ ]:
pptypes_str = ','.join( pptype2res.keys() )
pptypes_str

In [ ]:
fnf = pjoin(gv.data_dir,subdir_res,'df_collected_medcondsep_rest&entire_prescale,scale=medcond.pkl')

In [ ]:
df_noext = pd.concat(dfs, ignore_index=True)

read_saved = False
save = True

####################
import globvars as gv
subdir_res = 'collected'
#fnf = pjoin(gv.data_dir,subdir_res,'df_collected_medcondsep_rest&entire_prescale,scale=medcond.pkl')
#fnf = pjoin(gv.data_dir,subdir_res,'df_collected_medcondsep_rest&entire_noscaling=medcond.pkl')
#print(fnf)
#if read_saved:
    #df_noext = pd.read_pickle(fnf)
if save:
    #df_noext.to_pickle(fnf)
    
    temp_dir = pjoin( os.path.expandvars('$SCRATCH') , 'OSCBAGDIS', 'collected' )
    for ppt,opr in pptype2res.items():
        fn = f'{ppt}_output_per_raw.npz'
        fnf = pjoin(temp_dir, fn)
        print(fnf)
        np.savez(fnf, opr)
################

In [ ]:
dfs2 = []
#%debug
for dfc in dfs:
    #%debug
    from utils_postprocess_HPC import extendDf
    ppt = dfc['pptype']._values[0]
    #print(ppt)
    opr = pptype2res[ppt]
    dfs2 += [extendDf(dfc, opr)]
df = pd.concat(dfs2, ignore_index=True)
import gc; gc.collect()

In [ ]:
ppr?

In [ ]:
from pprint import pprint as ppr
ppr(opr, depth=1, compact=1)

In [ ]:
#df = extendDf( dfs[0], pptype2res['covmat_entire'] )

In [ ]:
df = extendDf( dfs[1], pptype2res['covmat_rest'] )

In [ ]:
dfc[['pptype','prefix', 'modtime']]

In [ ]:
r = 'S07_on merge_nothing basic covmat_entire onlyH_compl_subskip8right_onlyCB-copy_from_search_LFP'.split()
rn,g,it,ppt,pref = r

In [ ]:
opr[rn].keys()
#[pref].keys()

In [ ]:
prefixes

In [ ]:
dfc['prefix'].str.contains('@').any()

In [ ]:
#lbd(df.to_dict('records')[0])

In [ ]:
df_all = df

# Compare main stats with other feature sets

#### Create sub df

In [ ]:
df.columns

In [ ]:
### add LFP_side_to_use (in prev version was not put during extension of df)

In [ ]:
#df['LFP_side_to_use']

In [ ]:
from utils_postprocess_HPC import getMocFromRow, getMocFromRowMultiOPR

def lbd(row):
    moc = getMocFromRowMultiOPR(row,pptype2res)
    if moc is None:
        return None
#     pars = moc['pars']
#     s = f' ../run/run_ML.py'
#     for p,v in pars.items():
#         if p in ['iniAdd', 'search_best_LFP', 'code_ver' ]:
#             continue  
#         s += f' --{p} {v}'        
#     s += ' --exit_after artif_processed'

    s = moc['LFP_side_to_use_final']
    return s

#df['runs'] = df.apply(lbd,1)
#df['runs']

df['LFP_side_to_use'] = df.apply(lbd,1)
df['LFP_side_to_use']

#### Check number of pts stability

In [ ]:
#df

In [ ]:
df0 = df[ df['prefix'].str.startswith('onlyH_act_only1')  ]
#df0

In [ ]:
# n = 5
# for rowi, row in df0.iterrows():
#     from IPython import get_ipython; ipython = get_ipython()
#     ipython.run_line_magic('run', row['runs'] )
#     df0.loc[rowi, 'LFP_side_to_use'] = LFP_side_to_use
# #     n -= 1
# #     if n == 0:
# #         break

In [ ]:
df0['LFP_side_to_use'].nunique()

In [ ]:
grprn = df0.groupby( by=['pptype', 'rawname', 'interval_set','LFP_side_to_use'] )
grprnstd = grprn.std()
#grprnmn = grprn.mean()
display( grprn.size() )
# we would like to see zeros here
grprnstd['numpts']

In [ ]:
#opr['S01_off'][]
from utils_postprocess_HPC import multiLevelDict2TupleList
tpll = multiLevelDict2TupleList(opr,4,3)
tpll[0][-1]['pars']['best_LFP_info_file']

#### see basic numbers

In [ ]:
df['prefix_templ']

In [ ]:
use_prefix = False
#dfsub = df[['rawname','prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled']]
dfsub = df[['rawname','grouping','interval_set','pptype',
            'prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled','sens','spec',
           'tremor_det_perf']]

prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8','onlyH_mob_subskip8','onlyH_compl_subskip8','allb_beta_noH_subskip8','allb_gamma_noH_subskip8','allb_tremor_noH_subskip8',
        'LFPrel_noself_subskip8','modSrc_self_subskip8','modSrc_subskip8','modLFP_subskip8']

if use_prefix:
    prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8',
                        'onlyH_mob_subskip8','onlyH_compl_subskip8']
    prefix_role = 'prefix'
else:
    #pref_templs_to_compare = ['onlyH_subskip8%%','onlyH_act_subskip8%%',
    #                    'onlyH_mob_subskip8%%','onlyH_compl_subskip8%%',
    #                         'onlyH_modLFP_subskip8%%']
    pref_templs_to_compare = ['onlyH_subskip8@B-B','onlyH_act_subskip8@B-B',
                        'onlyH_mob_subskip8@B-B','onlyH_compl_subskip8@B-B',
                             'onlyH_modLFP_subskip8@B-B']
    prefix_role = 'prefix_templ'

dfsub2 = dfsub[ dfsub[prefix_role].isin(pref_templs_to_compare) ]

#dfsub2 = dfsub2[ (dfsub2['grouping'] == grp) & (dfsub2['interval_set'] == it) ]
grp =  dfsub2.groupby( by=['pptype', prefix_role,'grouping','interval_set'] )

cts = set( list( grp.count()['rawname'] ) ) 
assert tuple( cts)  == tuple([11]), cts


#print ( len( dfsub2.groupby('prefix') ) )
dfmn = grp.mean().reset_index()
dfstd = grp.std().reset_index()

rename_dict = dict( zip( list( dfstd.columns), [col + '_std' for col in  dfstd.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
del rename_dict['pptype']
dfstd = dfstd.rename(columns=rename_dict)

rename_dict = dict( zip( list( dfmn.columns), [col + '_mean' for col in  dfmn.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
del rename_dict['pptype']
dfmn = dfmn.rename(columns=rename_dict)

dfstat = dfmn
for col in dfstd.columns:
    if col in ['prefix_templ','prefix','num_std','grouping','interval_set']:
        continue
    print(col)
    dfstat[col] = dfstd[col]
cols_desired = ['pptype',prefix_role, 'grouping','interval_set','bacc_mean','bacc_std',
                      'tremor_det_perf_mean','tremor_det_perf_std', 'num_mean']
dfstat_short = dfstat[cols_desired]

display( dfstat_short )

In [ ]:
dfstat_short.query('grouping == "merge_nothing" and interval_set == "basic"')